In [776]:
import csv
import json
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import datetime


In [777]:
print("Wpływ COVID na świat kryptowalut")

Wpływ COVID na świat kryptowalut


In [778]:
#1.Porównywanie czy wraz ze wzrostem zachorowań wzrasta równiez ogólny poziom biedy
#2.Czy za każdym jak był wzrost zachorowań  to ceny kryptowalut się gwałtowanie zmieniały(procentowo)
#3.Jak ilość śmierci w 5 największych państwach wpłyneło na średnią pensję informatyków w pythonie w polsce
#4.Czy w tych państwach w których był największy covid podskoczyła liczba samobójstw i liczbę stwierdzonych depresji niż w tych w których było najmniej przypadków?
#5.Czy wielkość kraju miała wpływ na szybkość(rekord szybkości rozprzestrzeniania się )
#7. Czy w krajach w których jest starsze społeczeńśtwo i jest więcej szowinistów było więcej śmierci wśród męzczyzn
#8.Czy w krajach w których średni poziom inteligencji jest najwyższy było najwięcej szczepionek
#źródlo danych
# https://www.kaggle.com/datasets/sudalairajkumar/cryptocurrencypricehistory?select=coin_Bitcoin.csv

In [779]:
#przekształcenie danych stricte do wykresów dziennych i połączenie danych COVID i kursy cryptowalut(W doskonałej większości są one uzależnione od BTC(co pokażemy w następnych wykresach)
#IMPORT DANYCH Z COVID(bierzemy wszystkie dane)
#bierzemy tabelke oznaczoną jako new cases gdyż ona pokazuje nam ile jest przypadków nowego dnia(przez co możemy zobaczyć tzw. wybuchy zakażeń).
columnListCOVID=["Date","New cases"]
day_wise = pd.read_csv('day_wise.csv',usecols=columnListCOVID)
day_wise['Date'] = pd.to_datetime(day_wise['Date'])
day_wise['Date'] = day_wise['Date'].dt.date
print(type(day_wise))
#sprawdzamy czy dane są unikatowe
# print(day_wise.nunique())
#import danych z bitcoin, bierzemy wartość high (czyli tą za ile sprzedał się najwyżej danego dnia, w praktyce nie ma to większego znaczenia gdyż wachania dzienne są rzędu 0,05% co w tak długiej perspektywie nie ma znaczenia)
columnListBTC=["Date","High"]
bitCoinPrice= pd.read_csv('coin_BitCoin.csv',usecols=columnListBTC)
bitCoinPrice['Date']=pd.to_datetime(bitCoinPrice['Date'])
#we are stripping data from hours/minutes/seconds
bitCoinPrice['Date']=bitCoinPrice['Date'].dt.date
#bierzemy dane od 2020-01-22 do 2020-06-17
filterBTC=bitCoinPrice[(bitCoinPrice['Date'] >= pd.Timestamp('2020-01-22')) & (bitCoinPrice['Date'] <= pd.Timestamp('2020-06-17'))]
#sprawdzamy czy dane są unikatowe
# print(x.nunique())
print(type(filterBTC))
#łączymy dwie kolumny
merged_data=pd.merge(day_wise,filterBTC,how='inner',on='Date')
# print(len(merged_data))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


C:\Users\PrzemekZiemia\PycharmProjects\bigDataJupyter\venv\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [780]:
#now we are making plots
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig = px.bar(merged_data, x="Date", y="New cases", width=700, color_discrete_sequence=['#E11584'])
fig2 = px.line(merged_data, x="Date", y="High", width=700, color_discrete_sequence=['#FFC0CB'])
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Time"
subfig.layout.yaxis.title="BTC"
subfig.layout.yaxis2.title="New Cases"
subfig.update_layout(xaxis_rangeslider_visible=True)
subfig.show()

In [781]:
#Teraz zbadamy czy w tych krajach w których jest największy poziom inteligencji było najwięcej osób zaszczepionych(w procentach)

In [782]:
from plotly.offline import iplot
columnListVac=['country','people_vaccinated_per_hundred']
countryVac=pd.read_csv('country_vaccinations.csv',usecols=columnListVac)

mergCountryVac=countryVac.groupby(['country']).max()
#importujemy dane dot. inteligencji
columnListIQ=['country','iq']
cIQ=pd.read_csv('iqdata.csv',usecols=columnListIQ)
mergedIQVAC=pd.merge(cIQ,mergCountryVac,how='inner',on='country')
#pod uwagę będzie brane wszystkie pańśtwa

In [783]:
#we are now doing more graphs
import plotly.graph_objects as go
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = mergedIQVAC['iq'],
                   colorscale = 'Electric',
                   showscale = True),
        dimensions = list([
            dict(
                 label = "people_vaccinated", values = mergedIQVAC['people_vaccinated_per_hundred']),
            dict(
                 label = 'iq', values = mergedIQVAC['iq']),])
    )
)

fig.show()

In [784]:
import scipy
import seaborn as sns
#Sprawdzimy czy inne kryptowaluty(pokażemy dlaczego analizowalismy tylko BTC w pierwszym wykresie)
#BTC VS ETHEREUM VS Litecoin
columnListcrypto=["Symbol","Date","High"]
bitCoinPrice= pd.read_csv('coin_BitCoin.csv',usecols=columnListcrypto)
bitCoinPrice['Date']=pd.to_datetime(bitCoinPrice['Date'])
#eth
columnListBTC=["Symbol","Date","High"]
ETHPrice= pd.read_csv('coin_Ethereum.csv',usecols=columnListcrypto)
ETHPrice['Date']=pd.to_datetime(ETHPrice['Date'])
#litecoin
columnListcrypto=["Symbol","Date","High"]
liteCoinPrice= pd.read_csv('coin_Litecoin.csv',usecols=columnListcrypto)
liteCoinPrice['Date']=pd.to_datetime(liteCoinPrice['Date'])
merged_data=pd.merge(pd.merge(bitCoinPrice,ETHPrice,on='Date',how='inner'),liteCoinPrice,on='Date',how='inner')
merged_data.corr(method='kendall')
fig = px.imshow(merged_data.corr(method='pearson'),  labels=dict(x="Korelacja kryptowalut", y="Nazwa", color="Korelacja"),
                x=['BTC', 'ETH', 'LTC'],
                y=['BTC', 'ETH', 'LTC'], text_auto=True)
fig.show()

In [785]:
#Czy w pańśtwach w których wskaźnik przemocy wobec kobiet jest wysoki umarło więcej męzczyzn ?
#źródło:https://data.oecd.org/inequality/violence-against-women.htm

In [786]:
columnListCOVID=["country","deaths_male",'deaths_female']
deathsMalebe=pd.read_csv('Dataset-recent.csv',usecols=columnListCOVID)
deathsMale=deathsMalebe.groupby(['country']).sum()
deathsMale['Percentage']=deathsMale['deaths_male']*100/(deathsMale['deaths_male']+deathsMale['deaths_female'])
deathsMale=deathsMale[deathsMale.Percentage.notnull()]
columnListWoman=["country","womensDangerIndex"]
#wskaźnik ten jest opisano jako:Women’s Danger Index ranked each country based on eight factors: street safety for wojmen, intentional homicide of women, non-partner sexual violence, intimate partner sexual violence, legal discrimination, global gender gap, gender inequality index, and violence against women attitudes.
womanIndex=pd.read_csv('woman.csv',usecols=columnListWoman)
merged_data=pd.merge(deathsMale,womanIndex,how='inner',on='country')

In [787]:
#making graph
merged_data.sort_values(by=['womensDangerIndex'])
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig = px.line(merged_data, x="country", y="Percentage", width=700, color_discrete_sequence=['#E11584'])
fig2 = px.line(merged_data, x="country", y="womensDangerIndex", width=700, color_discrete_sequence=['#FFC0CB'])
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="country"
subfig.layout.yaxis.title="Percentage of death by male"
subfig.layout.yaxis2.title="Womens Danger Index"
subfig.update_layout(xaxis_rangeslider_visible=True)
subfig.show()